# Pedersen Commitments

**Module 09** | 09-commitments-sigma-protocols

*C = g^m * h^r, homomorphic, perfect hiding*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **pedersen commitments**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Commitment Schemes](09a-commitment-schemes.ipynb).
- Concepts and notation introduced in the previous notebook.

## Pedersen Commitment Scheme

In [ ]:
# Setup: group G of prime order q, generators g, h
# Commit: C = g^m * h^r
# Open: reveal m, r
p = next_prime(10^20)
g = Mod(primitive_root(p), p)
# h should be chosen so that log_g(h) is unknown
# TODO: set up Pedersen parameters

## Homomorphic Property

In [ ]:
# C1 * C2 = g^(m1+m2) * h^(r1+r2)
# Commitments can be added without opening!
# TODO: demonstrate homomorphic addition

## Perfect Hiding

In [ ]:
# For any m, the commitment C is uniformly random
# (because r is random and h^r is uniform)
# TODO: illustrate that different r values produce uniform C

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **pedersen commitments**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Sigma Protocols](09c-sigma-protocols-intuition.ipynb)